# Finding the origin of replication (Ori)

## Terminologies
*k-mer - fragment (substring) of the dna with the length of `k`

## 1.1 Hidden Messages in the Replication Origin
We'd like to find where does the replication origin given a dna string. To start, we're going to count what's the most frequent k-mer by counting the frequency.

In [1]:
def pattern_count(text, pattern):
    count = 0
    for i in range(len(text) - len(pattern) + 1):
        if text[i:i+len(pattern)] == pattern:
            count += 1
    return count

# Example usage
text = "GCGCG"
pattern = "GCG"
print(pattern_count(text, pattern))  # Output: 2

2


We say that Pattern is a most frequent k-mer in `text` if it maximizes Count(Text, Pattern) among all k-mers. To find what k-mer is the most frequent in the `text`, the most straightforward approach is to check all k-mers in the `text` string.

In [2]:
def frequent_words(text, k):
    freq_map = {}
    n = len(text)
    
    for i in range(n - k + 1):
        pattern = text[i:i+k]
        if pattern in freq_map:
            freq_map[pattern] += 1
        else:
            freq_map[pattern] = 1
    
    max_count = max(freq_map.values())
    most_frequent = [pattern for pattern, count in freq_map.items() if count == max_count]
    
    return most_frequent

# Example usage
text = "ACTGACTCCCACCCC"
k = 3
print(frequent_words(text, k))  # Output: ['GC', 'CG']

['CCC']


## 1.3 Some Hidden Messages are More Surprising than Others

Let's jump to other topic a bit. This seems unrelated for now, but we'll continue. 
Recall the complement strand of dna. The A most likely pair to T, G to C, and so on. Also recall that generally we write the dna string from 5' to 3', hence denoting the complement is reversal.
For example, the complement AAAACCCGGT is ACCGGGTTTT.

In [3]:
def reverse_complement(kmer):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    reverse_kmer = ''.join(complement[base] for base in reversed(kmer))
    return reverse_kmer

# Example usage
kmer = "AAAACCCGGT"
print(reverse_complement(kmer))  # Output: ACCGGGTTTT

ACCGGGTTTT


The reason we jump to the reverse complement talks is this:
Interestingly, among the four most frequent 9-mers in ori of Vibrio cholerae, ATGATCAAG and CTTGATCAT are reverse complements of each other, resulting in the six total occurrences of these strings shown below.
```
atcaatgatcaacgtaagcttctaagcATGATCAAGgtgctcacacagtttatccacaac ctgagtggatgacatcaagataggtcgttgtatctccttcctctcgtactctcatgacca cggaaagATGATCAAGagaggatgatttcttggccatatcgcaatgaatacttgtgactt gtgcttccaattgacatcttcagcgccatattgcgctggccaaggtgacggagcgggatt acgaaagcatgatcatggctgttgttctgtttatcttgttttgactgagacttgttagga tagacggtttttcatcactgactagccaaagccttactctgcctgacatcgaccgtaaat tgataatgaatttacatgcttccgcgacgatttacctCTTGATCATcgatccgattgaag atcttcaattgttaattctcttgcctcgactcatagccatgatgagctCTTGATCATgtt tccttaaccctctattttttacggaagaATGATCAAGctgctgctCTTGATCATcgtttc
```

Remember here we have circular DNA of a Vibrio cholerae, an eukaryote.
This observation leads us to the working hypothesis that ATGATCAAG and its reverse complement CTTGATCAT indeed represent DnaA boxes in Vibrio cholerae.

However, before concluding that we have found the DnaA box of Vibrio cholerae, the careful bioinformatician should check if there are other short regions in the Vibrio cholerae genome exhibiting multiple occurrences of ATGATCAAG (or CTTGATCAT). After all, maybe these strings occur as repeats throughout the entire Vibrio cholerae genome, rather than just in the ori region. To this end, we need to count the occurence of the kmer substring throughout the genome. The kmer substring is denoted with `pattern` in below code.

In [8]:
def retrieve_position_index(pattern, genome):
    positions = []
    pattern_length = len(pattern)
    genome_length = len(genome)
    
    for i in range(genome_length - pattern_length + 1):
        if genome[i:i+pattern_length] == pattern:
            positions.append(i)
    
    return ' '.join(map(str, positions))

# Example usage
genome = "GATATATGCATATACTT"
pattern = "ATAT"
print(retrieve_position_index(pattern, genome))  # Output: "1 3 9"

1 3 9


In [10]:
def read_genome_from_file(file_path):
    with open(file_path, 'r') as file:
        genome = file.read().strip()
    return genome

# Example usage
file_path = 'dataset/vibrio_cholerae_genome.txt'  # Replace with the path to your genome file
genome = read_genome_from_file(file_path)
pattern = "CTTGATCAT"
print(retrieve_position_index(pattern, genome))

60039 98409 129189 152283 152354 152411 163207 197028 200160 357976 376771 392723 532935 600085 622755 1065555


## 1.4 An Explosion of Hidden Messages

### The Clump Finding Problem
It turned out, finding Ori (origin of replication) is no trivial, especially when we switch to other organism.
Our next plan is to have a sliding window with substring length of L, which is basically a substring from genome. Then we count occurences of each k-mers within that substring. A k-mer categorized as `clump` if it appears at least `t` times. We repeat this until the sliding window ended.
```
find_clumps(text, k, L, t)

# where:
# text - genome
# k - the length of k-mer
# L - the length of substring
```

In [2]:
def find_clumps(text, k, L, t):
    clumps = set()
    n = len(text)
    
    for i in range(n - L + 1):
        window = text[i:i+L]
        freq_map = {}
        
        for j in range(L - k + 1):
            kmer = window[j:j+k]
            if kmer in freq_map:
                freq_map[kmer] += 1
            else:
                freq_map[kmer] = 1
        
        for kmer, count in freq_map.items():
            if count >= t:
                clumps.add(kmer)
    
    return clumps

# Example usage
text = "CGGACTCGACAGATGTGAAGAACGACAATGTGAAGACTCGACACGACAGAGTGAAGAGAAGAGGAAACATTGTAA"
k = 5
L = 50
t = 4
print(find_clumps(text, k, L, t))  # Output: {'CGACA' 'GAAGA'}

{'CGACA', 'GAAGA'}


In [3]:
def read_input_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        text = lines[0].strip()
        k, L, t = map(int, lines[1].strip().split())
    return text, k, L, t

# Example usage
file_path = 'dataset/code_challenge_clump_finding_problem.txt'  # Replace with the path to your input file
text, k, L, t = read_input_from_file(file_path)
print(find_clumps(text, k, L, t))

{'CTATAGTGGT', 'ATTACTGTAA', 'GGCAATGGCT', 'GTTATACGGG', 'GGACCAGCAG', 'CTGACTGCTG', 'ACGCCGGGCA', 'GCACGCCGGG', 'CACGCCGGGC'}
